In [99]:
from parity_task_feedback import parity_task_feedback

In [100]:
n_bits = 10
p_true = 0.3
n_train_samples = 50
n_test_samples = 20
true_bits = np.random.binomial(1, p_true, n_bits)
samples = np.random.binomial(1, 0.5, (n_train_samples + n_test_samples, n_bits))
masked_samples = samples * true_bits
parity = np.sum(masked_samples, axis=1) % 2
train_samples = samples[:n_train_samples]
train_parity = parity[:n_train_samples]

test_samples = samples[n_train_samples:]
test_parity = parity[n_train_samples:]
# Because algorithm is a string, we can’t call it directly. Instead, we can use eval to evaluate it as a Python expression.

print(test_parity)

try:
    predictions = algorithm(train_samples, train_parity, test_samples)
except Exception as e:
    print(e)

np.sum(predictions == test_parity) / n_test_samples


[0 0 1 1 1 1 0 1 1 0 1 0 0 0 0 1 0 0 1 1]
(50, 10)
(50,)
(50, 1)
all the input array dimensions for the concatenation axis must match exactly, but along dimension 0, the array at index 0 has size 50 and the array at index 1 has size 1


In [96]:


performance, message = utility_func(algorithm_str=algorithm_str)

In [118]:

from parity_task_feedback import parity_task_feedback


parity_task_feedback.utility.func, parity_task_feedback.utility.str = parity_task_feedback.utility.get_utility()
initial_solution = parity_task_feedback.get_solution()
print(initial_solution)
initial_solution = """import numpy as np

def algorithm(train_samples, train_parity, test_samples):
    n = train_samples.shape[1]
    aug_matrix = np.hstack((train_samples, train_parity.reshape(-1, 1))) # Fix here
    for i in range(n):
        pivot = np.where(aug_matrix[i:, i] == 1)[0]
        if len(pivot) > 0:
            pivot_row = pivot[0] + i
            if pivot_row != i:
                aug_matrix[[i, pivot_row]] = aug_matrix[[pivot_row, i]]
            for j in range(i + 1, len(aug_matrix)):
                if aug_matrix[j, i] == 1:
                    aug_matrix[j] = (aug_matrix[j] - aug_matrix[i]) % 2
    true_bits = np.zeros(n, dtype=int)
    for i in reversed(range(n)):
        true_bits[i] = aug_matrix[i, n]
        for j in range(i + 1, n):
            if aug_matrix[i, j] == 1:
                true_bits[i] ^= true_bits[j]
    test_parity = np.sum(true_bits + test_samples, axis=1) % 2
    return test_parity"""
print(parity_task_feedback.utility.func(initial_solution))
# parity_task_feedback.add_solution(improve_algorithm(parity_task_feedback.get_solution, parity_task_feedback.utility, language_model))

# print(parity_task_feedback.solutions)

import numpy as np

def algorithm(train_samples, train_parity, test_samples):
    n = train_samples.shape[1]
    aug_matrix = np.hstack((train_samples, train_parity.reshape(1, -1)))
    for i in range(n):
        pivot = np.where(aug_matrix[i:, i] == 1)[0]
        if len(pivot) > 0:
            pivot_row = pivot[0] + i
            if pivot_row != i:
                aug_matrix[[i, pivot_row]] = aug_matrix[[pivot_row, i]]
            for j in range(i + 1, len(aug_matrix)):
                if aug_matrix[j, i] == 1:
                    aug_matrix[j] = (aug_matrix[j] - aug_matrix[i]) % 2
    true_bits = np.zeros(n, dtype=int)
    for i in reversed(range(n)):
        true_bits[i] = aug_matrix[i, n]
        for j in range(i + 1, n):
            if aug_matrix[i, j] == 1:
                true_bits[i] ^= true_bits[j]
    test_parity = np.sum(true_bits + test_samples, axis=1) % 2
    return test_parity
(0.016666666666666666, '')


In [114]:
from helpers import extract_code

def improve_algorithm(initial_solution, utility, language_model):

    """Improves a solution according to a utility function."""
    
    expertise = "You are an expert computer science researcher and programmer, especially skilled at optimizing algorithms."

    debug_message = f"""You are given the following solution to a problem:
```python
{initial_solution}
```

You will be evaluated based on this score function:
```python
{utility.str}
```

The score of the current solution is {utility.func(initial_solution)[0]}.
Error messages / print statements returned from the current solution are: {utility.func(initial_solution)[1]}.

You must better understand the problems with the current solution by inserting print statements to debug the solution. The output of these print statements will be used to fix the solution using a language model."""
    debug_solutions = language_model.batch_prompt(expertise, [debug_message] * language_model.budget)
    debug_solutions = extract_code(debug_solutions)

    
    repair_message = f"""You are given the following solution to a problem:

```python
{initial_solution}
```

You will be evaluated based on this score function:
```python
{utility.str}
```

The score of the current solution is {utility.func(initial_solution)[0]}.
Error messages / print statements from the debug solution are: {utility.func(debug_solution)[1]}.

You must return an improved solution. Be as creative as you can under the constraints.
Your primary improvement must be novel and non-trivial. First, propose an idea, then implement it."""
    repair_solutions = language_model.batch_prompt(expertise, [repair_message.format(solution=debug_solution) for debug_solution in debug_solutions])
    repair_solutions = extract_code(repair_solutions)
    best_solution = max(repair_solutions, key=lambda x: utility.func(x))
    return best_solution

In [110]:
returns = ['\nimport numpy as np\n\ndef algorithm(train_samples, train_parity, test_samples):\n    print(f"train_samples shape: {train_samples.shape}")\n    print(f"train_parity shape: {train_parity.shape}")\n    print(f"test_samples shape: {test_samples.shape}")\n    \n    n = train_samples.shape[1]\n    \n    print(f"Reshaped train_parity shape: {train_parity.reshape(1, -1).shape}")\n    \n    try:\n        aug_matrix = np.hstack((train_samples, train_parity.reshape(1, -1)))\n    except Exception as e:\n        print(f"Error while creating aug_matrix: {e}")\n        return None\n    \n    for i in range(n):\n        pivot = np.where(aug_matrix[i:, i] == 1)[0]\n        if len(pivot) > 0:\n            pivot_row = pivot[0] + i\n            if pivot_row != i:\n                aug_matrix[[i, pivot_row]] = aug_matrix[[pivot_row, i]]\n            for j in range(i + 1, len(aug_matrix)):\n                if aug_matrix[j, i] == 1:\n                    aug_matrix[j] = (aug_matrix[j] - aug_matrix[i]) % 2\n                    \n    true_bits = np.zeros(n, dtype=int)\n    for i in reversed(range(n)):\n        true_bits[i] = aug_matrix[i, n]\n        for j in range(i + 1, n):\n            if aug_matrix[i, j] == 1:\n                true_bits[i] ^= true_bits[j]\n                \n    test_parity = np.sum(true_bits + test_samples, axis=1) % 2\n    return test_parity\n']
print(returns[0])


import numpy as np

def algorithm(train_samples, train_parity, test_samples):
    print(f"train_samples shape: {train_samples.shape}")
    print(f"train_parity shape: {train_parity.shape}")
    print(f"test_samples shape: {test_samples.shape}")
    
    n = train_samples.shape[1]
    
    print(f"Reshaped train_parity shape: {train_parity.reshape(1, -1).shape}")
    
    try:
        aug_matrix = np.hstack((train_samples, train_parity.reshape(1, -1)))
    except Exception as e:
        print(f"Error while creating aug_matrix: {e}")
        return None
    
    for i in range(n):
        pivot = np.where(aug_matrix[i:, i] == 1)[0]
        if len(pivot) > 0:
            pivot_row = pivot[0] + i
            if pivot_row != i:
                aug_matrix[[i, pivot_row]] = aug_matrix[[pivot_row, i]]
            for j in range(i + 1, len(aug_matrix)):
                if aug_matrix[j, i] == 1:
                    aug_matrix[j] = (aug_matrix[j] - aug_matrix[i]) % 2
                    
    tru

In [113]:
print(parity_task_feedback.utility.func(returns[0])[1])

train_samples shape: (80, 10)
train_parity shape: (80,)
test_samples shape: (20, 10)
Reshaped train_parity shape: (1, 80)
Error while creating aug_matrix: all the input array dimensions for the concatenation axis must match exactly, but along dimension 0, the array at index 0 has size 80 and the array at index 1 has size 1
train_samples shape: (80, 10)
train_parity shape: (80,)
test_samples shape: (20, 10)
Reshaped train_parity shape: (1, 80)
Error while creating aug_matrix: all the input array dimensions for the concatenation axis must match exactly, but along dimension 0, the array at index 0 has size 80 and the array at index 1 has size 1
train_samples shape: (80, 10)
train_parity shape: (80,)
test_samples shape: (20, 10)
Reshaped train_parity shape: (1, 80)
Error while creating aug_matrix: all the input array dimensions for the concatenation axis must match exactly, but along dimension 0, the array at index 0 has size 80 and the array at index 1 has size 1

